### well structured - an ode to databases


# linear regression

### import libraries

In [40]:
#import sqlite3
import pandas as pd

from sklearn.linear_model import LinearRegression

# import the visualization package: seaborn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

In [41]:
lm = LinearRegression()
lm

LinearRegression()

In [57]:
company_name_1 = 'iShares Global Timber & Forestry UCITS ETF USD Acc'
company_name_2 = 'Weyerhaeuser Company'

aux_var_stock_price_1 = pd.read_csv('data/' + company_name_1+ '.csv')
aux_var_stock_price_2 = pd.read_csv('data/' + company_name_1+ '.csv')

In [58]:
aux_var_stock_price_1 = aux_var_stock_price_1[['Close']]
aux_var_stock_price_2 = aux_var_stock_price_2[['Close']]

In [59]:
X = aux_var_stock_price_1[['Close']]
Y = aux_var_stock_price_2[['Close']]

In [60]:
lm.fit(X,Y)

LinearRegression()

In [61]:
Yhat=lm.predict(X)
Yhat[0:5]  

array([[30.70940018],
       [30.70560074],
       [30.66720009],
       [30.44179916],
       [30.74080086]])

In [62]:
lm.intercept_

array([7.10542736e-15])

In [63]:
lm.coef_

array([[1.]])

# multiple linear regression

In [64]:
company_name_3 = 'Svenska Cellulosa Aktiebolaget SCA (publ)'
company_name_4 = 'Suzano S.A.'

aux_var_stock_price_3 = pd.read_csv('data/' + company_name_3+ '.csv')
aux_var_stock_price_4 = pd.read_csv('data/' + company_name_4+ '.csv')

In [65]:
aux_var_stock_price_3 = aux_var_stock_price_3[['Close']]
aux_var_stock_price_4 = aux_var_stock_price_4[['Close']]
print(aux_var_stock_price_3)
print(aux_var_stock_price_4)

          Close
0    138.649994
1    135.500000
2    137.250000
3    134.949997
4    133.550003
..          ...
605  162.300003
606  160.149994
607  164.000000
608  163.000000
609  164.300003

[610 rows x 1 columns]
     Close
0     8.70
1     8.64
2     8.94
3     8.78
4     8.95
..     ...
600  12.47
601  12.41
602  12.66
603  12.83
604  12.78

[605 rows x 1 columns]


In [66]:
aux_var_stock_prices = pd.concat([aux_var_stock_price_2, aux_var_stock_price_3, aux_var_stock_price_4], axis=1)
print(aux_var_stock_prices)
Z = aux_var_stock_prices

         Close       Close  Close
0    30.709400  138.649994   8.70
1    30.705601  135.500000   8.64
2    30.667200  137.250000   8.94
3    30.441799  134.949997   8.78
4    30.740801  133.550003   8.95
..         ...         ...    ...
605  28.478600  162.300003    NaN
606  28.685301  160.149994    NaN
607  28.638100  164.000000    NaN
608  28.737900  163.000000    NaN
609        NaN  164.300003    NaN

[610 rows x 3 columns]


In [67]:
lm.fit(Z, X)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

# evaluation by visualization

In [ ]:
width = 12
height = 10
plt.figure(figsize=(width, height))
sns.regplot(x="highway-mpg", y="price", data=df)
plt.ylim(0,)